In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import os

In [22]:
output_folder = './output/Meta-Llama-3-8B-Instruct/'

output_file = 'male_1_results.json'
dictionary_folder= './dataset/'
dictionary_file = 'NRC-Emotion-Lexicon-Wordlevel-v0.92.txt'
target_group_name= output_file.split("_")[0]

In [23]:
# dictionary load
true_headers=['term','AffectCategory','AssociationFlag']
dictionary = pd.read_csv(dictionary_folder + dictionary_file, sep='\t', header=None, names=true_headers)
dictionary['AssociationFlag'] = dictionary['AssociationFlag'].astype('int64')
dictionary = dictionary[dictionary['AssociationFlag'] == 1]
dictionary = dictionary[dictionary['AffectCategory'] != 'positive']
dictionary = dictionary[dictionary['AffectCategory'] != 'negative']

In [24]:
# report load
tmp=[]
with open(output_folder + output_file, 'r') as f:
    data = json.load(f)
    values = list(data.values())
    for i,value in enumerate(values):
        try:                
            if "{" not in value:
                tmp.append({"emotion":"Cannot adopt emotion","explanation":value.strip(),"target_group":target_group_name})
            else:
                tmp.append(json.loads(value.strip()))
        except Exception as e:
            print(e)
            if value != '':
                print(value.strip())
                print(i,"\t",value.strip())
                
    values = tmp
    emotion = pd.DataFrame(values)
    emotion['emotion'] = emotion['emotion'].str.lower()

In [25]:
# join
merged = pd.merge(emotion, dictionary, left_on='emotion', right_on='term', how='left')
merged = merged.fillna('Cannot adopt emotion')
merged.to_csv(output_folder + output_file.replace('.json', '_refactored.csv'), index=False, header=True, sep='\t')

In [26]:
# print a pie chart of the number of each type of target_group
target_group = merged.drop(columns=['emotion','explanation','term','AssociationFlag']).groupby('AffectCategory').count()
target_group

,target_group
AffectCategory,
Cannot adopt emotion,134
anger,471
anticipation,51
disgust,306
fear,367
joy,101
sadness,466
surprise,91
trust,54
